# Get lugar de ocurrencia

In [3]:
pip install unidecode

  Using cached Unidecode-1.3.6-py3-none-any.whl (235 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import re
import unidecode
from geopy.geocoders import GoogleV3

Clean the location dataset

In [9]:
df = pd.read_excel("data/dataset_cleaned.xlsx")
df

In [7]:
df["MOD_Lugar_Ocurrencia"] = df["De_Lugar_Ocurrencia"].str.lower()

In [8]:
def remover_acentos(x):
    """Creamos la funcion para remover los acentos"""
    if isinstance(x, str):
        return unidecode.unidecode(x)
    else:
        return x

# Aplicar la función a la columna "DE_LUGAR_OCURRENCIA"
df["De_Lugar_Ocurrencia"] = df["De_Lugar_Ocurrencia"].apply(remover_acentos)
df

,Codigo_Siniestro,Codigo_Cliente,Fe_Ocurrencia,Codigo_Poliza,Cd_Provincia,Cd_Ciudad,De_Lugar_Ocurrencia,Nu_Siniestro_Relacionado,In_Fraude,De_Causa_Siniestro,...,Color,Marca,Modelo,Año,Tipo_Transmision,ano,Clase,Cantidad_Parque_Vehicular,Tipo_Dia,dia_feriado
0,1,7668,2015-05-29,1,29.0,1.0,siniestro reportado via cover.,,0,Accidente/Choque,...,Blanco,TOYOTA,CAMRY,2001.0,CE - Automatico,2015,Automovil,773019,Weekday,0
1,2,23643,2016-02-12,2,29.0,1.0,c/ mayaguez urb. villas dumit,,0,Accidente/Choque,...,Rojo,KIA,SPORTAGE,2015.0,4x4 - Automatico,2016,Jeep,386706,Weekday,0
2,3,29730,2016-02-09,3,1.0,1.0,barahora,,0,Responsabilidad Civil Personas o Cosas,...,Blanco,DAIHATSU,DELTA,2007.0,Sincronico,2016,Carga,405292,Weekday,0
3,4,14301,2016-02-15,4,1.0,1.0,arroyo hondo ii,,0,Robo o Hurto Accesorios/ Partes o Piezas,...,NaN,NaN,NaN,NaN,NaN,2016,Jeep,386706,Weekday,0
4,5,24144,2016-02-03,5,1.0,1.0,avenida gustavo mejia ricart,,0,Accidente/Choque,...,Blanco,TOYOTA,PRADO,2013.0,%Sincronico,2016,Jeep,386706,Weekday,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18738,18702,192804,2023-04-04,11379,1.0,1.0,autopista duarte,,0,Rotura de Vidrios y/o Parabrisas,...,Blanco,JEEP,WRANGLER,2023.0,Renegade 4x4 - Automatico,2023,Jeep,653866,Weekday,0
18739,18703,181905,2023-06-03,11470,1.0,1.0,los multis los alcarrizos,,0,Rotura de Vidrios y/o Parabrisas,...,Blanco,ISUZU,D-MAX,2015.0,Automatico,2023,Jeep,653866,Weekend,0
18740,18704,182109,2023-05-22,11471,1.0,1.0,autopista duarte,,0,Rotura de Vidrios y/o Parabrisas,...,Azul,HYUNDAI,TUCSON,2014.0,GL - Automatico,2023,Jeep,653866,Weekday,0
18741,18705,302838,2023-06-01,11472,1.0,1.0,calle camino chiquito,,0,Accidente/Choque,...,Gris,TOYOTA,HILUX,2023.0,Cabina Sencilla Pick-Up 4x4 - Sincronico,2023,Jeep,653866,Weekday,0


In [ ]:
#rows=[{'avg_lat':52.509669, 'avg_lon':13.376294}]

def get_component(location, component_type):
    for component in location.raw['address_components']:
        if component_type in component['types']:
            return component['long_name']

#location.raw

#df_geo_unica = df_geo_unica.iloc[:20]

post_code_list = []
state_list = []
neighbourhood_list = []
locality_list = []
street_number_list = []
street_name_list = []

for i, j in zip(df_geo_unica["Y"], df_geo_unica['X']):
    location = geolocator.reverse((i,j), timeout = 500,  exactly_one=True)
    post_code = get_component(location, 'postal_code') #funciono
    post_code_list.append(post_code)
    
df_geo_unica['post_code'] = post_code_list
df_geo_unica.head(5)